In [1]:
import sys, os

# Move to project root
root = os.path.abspath("..")
sys.path.append(root)


from Utils.chunk_vis import visualize_chunks
from Utils.docs_chunker import splitter

In [45]:
import os 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

if PINECONE_API_KEY and PINECONE_INDEX_NAME and PINECONE_NAMESPACE:
    print("Pinecone variables are set.")
if OPENAI_API_KEY:
    print("OpenAI API key is set.")

Pinecone variables are set.
OpenAI API key is set.


In [2]:
import os
Books_dir = os.path.join(os.getcwd(), "Cleaned")
list = os.listdir(Books_dir)
print(list)

['History Class 10_cleaned.md', 'History Class 11 (Part 1)_cleaned.md', 'History Class 12 (Part 1.1)_cleaned.md', 'History Class 12 (Part 2.1)_cleaned.md', 'History Class 12 (Part 3.1)_cleaned.md', 'History Class 6 (Part 1)_cleaned.md', 'History Class 7_cleaned.md', 'History Class 8_cleaned.md', 'History Class 9_cleaned.md']


In [91]:
selected_file = Books_dir
for idx , book in enumerate(list):
    print(f"{idx} : {book}")

book_idx = int(input("Select the book index to chunk and visualize: "))
print(f"Selected book: {list[book_idx]}")
selected_file = os.path.join(Books_dir, list[book_idx])

0 : History Class 10_cleaned.md
1 : History Class 11 (Part 1)_cleaned.md
2 : History Class 12 (Part 1.1)_cleaned.md
3 : History Class 12 (Part 2.1)_cleaned.md
4 : History Class 12 (Part 3.1)_cleaned.md
5 : History Class 6 (Part 1)_cleaned.md
6 : History Class 7_cleaned.md
7 : History Class 8_cleaned.md
8 : History Class 9_cleaned.md
Selected book: History Class 12 (Part 3.1)_cleaned.md


In [92]:
chunker = splitter
with open(selected_file, "r", encoding="utf-8") as f:
    text = f.read()

chunks = chunker.split_text(text)

In [93]:
visualize_chunks(chunks[:6])

In [94]:
def chunk_stats(chunks):
    total_tables = 0

    max_table_idx = None
    min_table_idx = None

    max_chunk_idx = None
    min_chunk_idx = None

    avg_table_size = None  # <-- FIX

    for idx, chunk in enumerate(chunks):
        chunk_length = len(chunk)
        is_table = "<table" in chunk.lower()

        # Track largest/smallest chunk
        if max_chunk_idx is None or chunk_length > len(chunks[max_chunk_idx]):
            max_chunk_idx = idx

        if min_chunk_idx is None or chunk_length < len(chunks[min_chunk_idx]):
            min_chunk_idx = idx

        # Track tables
        if is_table:
            total_tables += 1

            if max_table_idx is None or chunk_length > len(chunks[max_table_idx]):
                max_table_idx = idx

            if min_table_idx is None or chunk_length < len(chunks[min_table_idx]):
                min_table_idx = idx

    print("\nTable Chunk Statistics:")
    print("-----------------------")
    print("Total Chunks:", len(chunks))
    print("Total Tables Found:", total_tables)

    print(f"\nLargest chunk  (Index: {max_chunk_idx}, Length: {len(chunks[max_chunk_idx])})")
    print(f"Smallest chunk (Index: {min_chunk_idx}, Length: {len(chunks[min_chunk_idx])})")

    if total_tables > 0:
        table_lengths = [len(chunk) for chunk in chunks if "<table" in chunk.lower()]
        avg_table_size = sum(table_lengths) / total_tables

        print(f"\nLargest table chunk  (Index: {max_table_idx}, Length: {len(chunks[max_table_idx])})")
        print(f"Smallest table chunk (Index: {min_table_idx}, Length: {len(chunks[min_table_idx])})")
        print(f"Average table chunk size: {avg_table_size}")

    return {
        "total_chunks": len(chunks),
        "total_tables": total_tables,
        "max_chunk_idx": max_chunk_idx,
        "min_chunk_idx": min_chunk_idx,
        "max_table_idx": max_table_idx,
        "min_table_idx": min_table_idx,
        "max_table_size": len(chunks[max_table_idx]) if max_table_idx is not None else None,
        "min_table_size": len(chunks[min_table_idx]) if min_table_idx is not None else None,
        "avg_table_size": avg_table_size   # <-- safe now
    }

stats = chunk_stats(chunks)



Table Chunk Statistics:
-----------------------
Total Chunks: 167
Total Tables Found: 5

Largest chunk  (Index: 154, Length: 1886)
Smallest chunk (Index: 93, Length: 348)

Largest table chunk  (Index: 154, Length: 1886)
Smallest table chunk (Index: 119, Length: 1342)
Average table chunk size: 1559.0


In [95]:
maxi = stats['max_chunk_idx']
mini = stats['min_chunk_idx']
print("\nVisualizing Largest Chunk:")



Visualizing Largest Chunk:


In [90]:
print("Max Chunk below")
visualize_chunks(chunks[stats["max_chunk_idx"]: stats["max_chunk_idx"] +1])
print("Min Chunk below")
visualize_chunks(chunks[stats["min_chunk_idx"]: stats["min_chunk_idx"] +1])

Max Chunk below


Min Chunk below


In [96]:
newChunks = []
if (len(chunks[mini]) > 200):
    newChunks = chunks
else:
    idx = 0
    while idx < len(chunks):
        newChunks.append(chunks[idx])
        if len(chunks[idx]) < 200:
            visualize_chunks(newChunks[max(idx - 2 , 0): idx])
            visualize_chunks(chunks[idx:idx + 3])
            left = int(input("Enter number of chunks to merge on the left (0 for none): "))
            right = int(input("Enter number of chunks to merge on the right (0 for none): "))

            currChunk = ""
            for i in range(idx - left , idx):
                if i >= 0:
                    currChunk += chunks[i] + "\n"
            for i in range(idx , idx + right + 1):
                if i < len(chunks):
                    currChunk += chunks[i] + "\n"

            for i in range(0 , left + 1):
                newChunks.pop()
            newChunks.append(currChunk)
            if right > 0:
                idx += right
        idx += 1

In [97]:
chunk_stats(newChunks)


Table Chunk Statistics:
-----------------------
Total Chunks: 167
Total Tables Found: 5

Largest chunk  (Index: 154, Length: 1886)
Smallest chunk (Index: 93, Length: 348)

Largest table chunk  (Index: 154, Length: 1886)
Smallest table chunk (Index: 119, Length: 1342)
Average table chunk size: 1559.0


{'total_chunks': 167,
 'total_tables': 5,
 'max_chunk_idx': 154,
 'min_chunk_idx': 93,
 'max_table_idx': 154,
 'min_table_idx': 119,
 'max_table_size': 1886,
 'min_table_size': 1342,
 'avg_table_size': 1559.0}

In [99]:
BOOK_NAME = "History Class 12 (Part 3)"  
print(f"Book Name: {BOOK_NAME}")

Book Name: History Class 12 (Part 3)


In [100]:
print(len(newChunks))

167


In [101]:
import os
import uuid
from tqdm import tqdm 
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv

# 1. Load Env & Config
load_dotenv()

PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
BATCH_SIZE = 100 

# 2. Initialize Clients
client = OpenAI()
pc = Pinecone()
index = pc.Index(PINECONE_INDEX_NAME)

# 3. Check if newChunks exists
if 'newChunks' not in locals() or not newChunks:
    raise ValueError("The variable 'newChunks' is missing or empty!")

print(f"Total chunks to process: {len(newChunks)}")

# 4. Process in Batches
# tqdm will now show a text-based progress bar (e.g., [===>   ] 20%)
for i in tqdm(range(0, len(newChunks), BATCH_SIZE), desc="Uploading Batches"):
    # Get current batch
    batch_end = min(i + BATCH_SIZE, len(newChunks))
    batch = newChunks[i:batch_end]
    
    try:
        # Generate Embeddings
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=batch
        )

        # Prepare Vectors
        vectors = []
        for chunk_text, emb_data in zip(batch, response.data):
            vector_id = str(uuid.uuid4())
            vectors.append({
                "id": vector_id,
                "values": emb_data.embedding,
                "metadata": {
                    "text": chunk_text,
                    "book_name": BOOK_NAME,
                    "chunk_index": i + batch.index(chunk_text)
                }
            })

        # Upsert to Pinecone
        index.upsert(vectors=vectors, namespace=PINECONE_NAMESPACE)

    except Exception as e:
        print(f"❌ Error on batch {i}: {e}")

print("✅ Upload complete.")

# 5. Quick Verification
stats = index.describe_index_stats()
print("\nIndex Stats:")
print(stats)

Total chunks to process: 167


Uploading Batches: 100%|██████████| 2/2 [00:13<00:00,  6.50s/it]


✅ Upload complete.

Index Stats:
{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '184',
                                    'content-type': 'application/json',
                                    'date': 'Tue, 09 Dec 2025 17:13:03 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '39',
                                    'x-pinecone-request-id': '1982987925042349216',
                                    'x-pinecone-request-latency-ms': '39'}},
 'dimension': 1536,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'default': {'vector_count': 2126}},
 'storageFullness': 0.0,
 'total_vector_count': 2126,
 'vector_type': 'dense'}


In [ ]:
import os
from openai import OpenAI
from pinecone import Pinecone

# 1. Configuration
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

client = OpenAI()
pc = Pinecone()
index = pc.Index(PINECONE_INDEX_NAME)

def query_pinecone(user_query, top_k=3):
    print(f"Querying for: '{user_query}'...")

    # 1. Generate embedding for the query
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=user_query
    )
    query_vector = response.data[0].embedding

    # 2. Query Pinecone
    # We filter by book_name to ensure we only search the relevant book if needed
    search_results = index.query(
        namespace=PINECONE_NAMESPACE,
        vector=query_vector,
        top_k=top_k,
        include_metadata=True,
        # filter={"book_name": "History Class 6"} # Optional: Add this if you have multiple books
    )


    # 3. Print Results
    print(f"\nFound {len(search_results['matches'])} matches:\n")
    
    for i, match in enumerate(search_results['matches']):
        score = match['score']
        metadata = match['metadata']
        text_content = metadata.get('text', 'No text found')
        
        print(f"--- Match {i+1} (Score: {score:.4f}) ---")
        print(f"Content: {text_content}")
        print("-" * 50)

    return search_results


In [52]:
visualize_chunks(results)

KeyError: '0 is not a valid field'